# **Make the master results files in this notebook**

* see ~/Documents/school/thesis/plots_and_results/label_predictions/get_classes.py for an example of how you did this before
    * kidding, that just gathers all the clusters

In [2]:
import pandas as pd

In [6]:
datasets=['cb','dg','jam', 'tm', 'li_crc','llc', 'vg', 'peng']
cluster_types = ['seurat']



for dataset in datasets:
    predictions = pd.DataFrame(columns=['dataset','cluster', 'truth', 'cibersort','gsea','gsva','metaneighbor','ora'])
    for clustering in cluster_types:
        #read the prediction scores
        ####df = pd.read_csv(f"/home/erik/Documents/school/thesis/program/output/{dataset}_results/BENCHMARK/MERGED_TABLES/{dataset}_{clustering}_fqn.PERMUTE.listToGetAuc.merged.tsv", sep='\t')
        if clustering == "seurat":
            df = pd.read_csv(f"./program/seurat_output/{dataset}_results/BENCHMARK/MERGED_TABLES/{dataset}_{clustering}_fqn.PERMUTE.listToGetAuc.merged.tsv", sep='\t')
        else:
            df = pd.read_csv(f"~./program/output/{dataset}_results/BENCHMARK/MERGED_TABLES/{dataset}_{clustering}_fqn.PERMUTE.listToGetAuc.merged.tsv", sep='\t')
        df = df.fillna(0)
        #make separate columns for class, cluster, and cluster number
        df['class'] = df.apply(lambda row: row['ClusterId_ClassId'].split('.', 2)[2], axis=1)
        df['cluster'] = df.apply(lambda row: row['ClusterId_ClassId'].split('.')[0], axis=1)
        df['clusternum'] = df.apply(lambda row: int(row['ClusterId_ClassId'].split('.')[0][-2:].replace('t', '0')), axis=1)

        method_list = ['CIBERSORT_BINARY__100__1','GSEA__100__1','GSVA__100__1','METANEIGHBOR_BINARY__100__1','ORA__100__1']
        for i in list(set(df['clusternum'])):
            #make an empty list and add the cluster number and truth label
            prediction_list = []
            prediction_list.append(f'{dataset}_{clustering}')
            prediction_list.append(i)
            cluster_preds = df[df['clusternum']==i]
            prediction_list.append(cluster_preds.loc[cluster_preds['Label'].idxmax()]['class'])
            #select the predictions for this cluster
            #add the predicted class for each method to the list
            for method in method_list:
                prediction_list.append(cluster_preds.loc[cluster_preds[method].idxmax()]['class'])
            #add the predictions for this cluster to our dataframe
            predictions = predictions.append(pd.Series(prediction_list, index=predictions.columns ), ignore_index=True)
        #add adobo
        ####df = pd.read_csv(f'/home/erik/Documents/school/thesis/new_methods/adobo/{dataset}/paper_fqn_results.tsv', sep='\t')
        ####df = pd.read_csv(f'/home/erik/Documents/school/thesis/new_methods/adobo/{dataset}/paper_standard_results.tsv', sep='\t')
        ####predictions['adobo'] = df['cell type']
        #add sccatch
            # this doesn't work quite right because of what i'm currently doing with the cluster numbers. 
            # scCatch still gets the filtered clusters, which makes it hard to match up since it doesn't leave a blank for unknowns
        ####df = pd.read_csv(f'/home/erik/Documents/school/thesis/new_methods/scCatch/{dataset}/paper_counts_results.tsv', sep='\t')
        ####predictions['sccatch'] = predictions.apply(lambda row: df[df.cluster==row['cluster']]['cell_type'].values,axis=1)
        predictions.to_csv(f'./seurat_predictions/{dataset}_{clustering}_predictions.tsv', sep='\t',index=False)
        display(predictions)


,dataset,cluster,truth,cibersort,gsea,gsva,metaneighbor,ora
0,cb_seurat,1,TCELL,TCELL,TCELL,TCELL,TCELL,TCELL
1,cb_seurat,2,TUMOR,MYELOID,BCELL,MYELOID,STROMAL,BCELL
2,cb_seurat,3,TUMOR,STROMAL,BCELL,STROMAL,STROMAL,BCELL
3,cb_seurat,4,TUMOR,BCELL,BCELL,STROMAL,BCELL,BCELL
4,cb_seurat,5,TUMOR,BCELL,BCELL,MYELOID,BCELL,BCELL
5,cb_seurat,6,TUMOR,BCELL,BCELL,BCELL,TCELL,BCELL
6,cb_seurat,7,BCELL,BCELL,BCELL,BCELL,BCELL,BCELL
7,cb_seurat,8,TUMOR,TCELL,BCELL,TCELL,TCELL,BCELL
8,cb_seurat,9,MYELOID,STROMAL,BCELL,TCELL,TCELL,BCELL
9,cb_seurat,10,TUMOR,TCELL,BCELL,MYELOID,TCELL,BCELL


,dataset,cluster,truth,cibersort,gsea,gsva,metaneighbor,ora
0,dg_seurat,1,IMMUNE,IMMUNE,IMMUNE,IMMUNE,VASCULAR,IMMUNE
1,dg_seurat,2,IMMUNE,IMMUNE,IMMUNE,IMMUNE,OLIGODENDROCYTE,IMMUNE
2,dg_seurat,3,IMMUNE,IMMUNE,IMMUNE,IMMUNE,OLIGODENDROCYTE,IMMUNE
3,dg_seurat,4,OPC,OPC,OPC,OPC,VASCULAR,OPC
4,dg_seurat,5,NEOPLASTIC,IMMUNE,NEOPLASTIC,ASTOCYTE,ASTOCYTE,NEOPLASTIC
5,dg_seurat,6,NEOPLASTIC,IMMUNE,NEOPLASTIC,IMMUNE,ASTOCYTE,NEOPLASTIC
6,dg_seurat,7,NEOPLASTIC,IMMUNE,NEOPLASTIC,OPC,ASTOCYTE,NEOPLASTIC
7,dg_seurat,8,IMMUNE,IMMUNE,IMMUNE,IMMUNE,IMMUNE,IMMUNE
8,dg_seurat,9,NEOPLASTIC,IMMUNE,NEOPLASTIC,NEURON,NEURON,NEOPLASTIC
9,dg_seurat,10,IMMUNE,IMMUNE,IMMUNE,IMMUNE,OLIGODENDROCYTE,IMMUNE


,dataset,cluster,truth,cibersort,gsea,gsva,metaneighbor,ora
0,jam_seurat,1,T.CD8,T.CD8,T.CD8,T.CD8,T.CD8,T.CD8
1,jam_seurat,2,T.CD4,T.CD4,T.CD4,T.CD4,T.CD8,T.CD4
2,jam_seurat,3,B.CELL,T.CD4,B.CELL,B.CELL,B.CELL,B.CELL
3,jam_seurat,4,T.CELL,T.CD4,T.CD4,T.CD4,T.CD8,T.CD4
4,jam_seurat,5,T.CD8,T.CD8,T.CD8,T.CD8,T.CD8,T.CD8
5,jam_seurat,6,MAL,CAF,ENDO.,CAF,CAF,ENDO.
6,jam_seurat,7,MACROPHAGE,MACROPHAGE,MACROPHAGE,MACROPHAGE,MACROPHAGE,MACROPHAGE
7,jam_seurat,8,MAL,MAL,B.CELL,MACROPHAGE,MACROPHAGE,MACROPHAGE
8,jam_seurat,9,MAL,T.CD4,ENDO.,MAL,CAF,ENDO.
9,jam_seurat,10,MAL,CAF,NK,CAF,CAF,CAF


,dataset,cluster,truth,cibersort,gsea,gsva,metaneighbor,ora
0,tm_seurat,1,BCELL,TCELL,TCELL,TCELL,TCELL,TCELL
1,tm_seurat,2,BCELL,TCELL,TCELL,TCELL,TCELL,TCELL
2,tm_seurat,3,BCELL,BCELL,BCELL,BCELL,BCELL,BCELL
3,tm_seurat,4,MELANOMA,BCELL,CAF,CAF,CAF,BCELL
4,tm_seurat,5,BCELL,TCELL,TCELL,TCELL,TCELL,TCELL
5,tm_seurat,6,MELANOMA,CAF,CAF,CAF,CAF,BCELL
6,tm_seurat,7,MELANOMA,MACROPHAGE,CAF,CAF,MACROPHAGE,CAF
7,tm_seurat,8,MELANOMA,BCELL,CAF,CAF,CAF,BCELL
8,tm_seurat,9,MACROPHAGE,MACROPHAGE,MACROPHAGE,MACROPHAGE,MACROPHAGE,MACROPHAGE
9,tm_seurat,10,BCELL,TCELL,TCELL,TCELL,TCELL,TCELL


,dataset,cluster,truth,cibersort,gsea,gsva,metaneighbor,ora
0,li_crc_seurat,1,EPITHELIAL,EPITHELIAL,EPITHELIAL,BCELL,BCELL,EPITHELIAL
1,li_crc_seurat,2,EPITHELIAL,EPITHELIAL,EPITHELIAL,EPITHELIAL,TCELL,EPITHELIAL
2,li_crc_seurat,3,EPITHELIAL,EPITHELIAL,EPITHELIAL,FIBROBLAST,FIBROBLAST,EPITHELIAL
3,li_crc_seurat,4,EPITHELIAL,EPITHELIAL,EPITHELIAL,EPITHELIAL,EPITHELIAL,EPITHELIAL
4,li_crc_seurat,5,EPITHELIAL,EPITHELIAL,EPITHELIAL,MACROPHAGE,MACROPHAGE,EPITHELIAL
5,li_crc_seurat,6,TCELL,TCELL,TCELL,BCELL,TCELL,TCELL
6,li_crc_seurat,7,MACROPHAGE,EPITHELIAL,EPITHELIAL,FIBROBLAST,FIBROBLAST,BCELL


,dataset,cluster,truth,cibersort,gsea,gsva,metaneighbor,ora
0,llc_seurat,1,T_CELLS,MACROPHAGES,DENDRITIC_CELLS,DENDRITIC_CELLS,MAST_CELLS,MACROPHAGES
1,llc_seurat,2,T_CELLS,MACROPHAGES,MACROPHAGES,MACROPHAGES,CANCER_CELLS,MACROPHAGES
2,llc_seurat,3,MACROPHAGES,MACROPHAGES,MACROPHAGES,MACROPHAGES,MAST_CELLS,MACROPHAGES
3,llc_seurat,4,T_CELLS,MACROPHAGES,DENDRITIC_CELLS,DENDRITIC_CELLS,CANCER_CELLS,MACROPHAGES
4,llc_seurat,5,T_CELLS,MACROPHAGES,MACROPHAGES,DENDRITIC_CELLS,MAST_CELLS,MACROPHAGES
5,llc_seurat,6,CANCER_CELLS,CANCER_CELLS,FIBROBLASTS,T_CELLS,MAST_CELLS,CANCER_CELLS
6,llc_seurat,7,B_CELLS,B_CELLS,B_CELLS,B_CELLS,MACROPHAGES,B_CELLS
7,llc_seurat,8,T_CELLS,MACROPHAGES,DENDRITIC_CELLS,DENDRITIC_CELLS,CANCER_CELLS,MACROPHAGES
8,llc_seurat,9,MACROPHAGES,MACROPHAGES,MACROPHAGES,MACROPHAGES,DENDRITIC_CELLS,MACROPHAGES
9,llc_seurat,10,T_CELLS,CANCER_CELLS,MACROPHAGES,DENDRITIC_CELLS,MAST_CELLS,MACROPHAGES


,dataset,cluster,truth,cibersort,gsea,gsva,metaneighbor,ora
0,vg_seurat,2,HSC_PROG_LIKE,MYELOID_LIKE,HSC_PROG,HSC_PROG_LIKE,HSC_PROG_LIKE,HSC_PROG
1,vg_seurat,3,MYELOID,MYELOID_LIKE,MYELOID,MYELOID,MYELOID,MYELOID
2,vg_seurat,4,HSC_PROG,HSC_PROG,HSC_PROG,HSC_PROG,HSC_PROG,HSC_PROG
3,vg_seurat,6,MYELOID,MYELOID_LIKE,HSC_PROG_LIKE,MYELOID,MYELOID,MYELOID_LIKE
4,vg_seurat,7,HSC_PROG_LIKE,HSC_PROG,HSC_PROG,HSC_PROG_LIKE,HSC_PROG_LIKE,HSC_PROG
5,vg_seurat,8,MYELOID_LIKE,MYELOID_LIKE,MYELOID_LIKE,MYELOID_LIKE,MYELOID_LIKE,MYELOID_LIKE
6,vg_seurat,9,HSC_PROG_LIKE,HSC_PROG_LIKE,HSC_PROG,HSC_PROG_LIKE,HSC_PROG_LIKE,HSC_PROG
7,vg_seurat,11,GMP_LIKE,GMP_LIKE,HSC_PROG,GMP_LIKE,GMP_LIKE,GMP_LIKE
8,vg_seurat,13,GMP_LIKE,MYELOID_LIKE,HSC_PROG_LIKE,GMP_LIKE,GMP_LIKE,MYELOID_LIKE
9,vg_seurat,15,GMP,GMP,GMP_LIKE,GMP,GMP,GMP


,dataset,cluster,truth,cibersort,gsea,gsva,metaneighbor,ora
0,peng_seurat,1,STELLATE_CELL,STELLATE_CELL,B_CELL,STELLATE_CELL,STELLATE_CELL,STELLATE_CELL
1,peng_seurat,2,MACROPHAGE,MACROPHAGE,B_CELL,MACROPHAGE,MACROPHAGE,MACROPHAGE
2,peng_seurat,3,ENDOTHELIAL_CELL,ENDOTHELIAL_CELL,B_CELL,ENDOTHELIAL_CELL,ENDOTHELIAL_CELL,ENDOTHELIAL_CELL
3,peng_seurat,4,DUCTAL_CELL_TYPE_2,DUCTAL_CELL_TYPE_2,B_CELL,DUCTAL_CELL_TYPE_2,DUCTAL_CELL_TYPE_2,DUCTAL_CELL_TYPE_2
4,peng_seurat,5,DUCTAL_CELL_TYPE_1,DUCTAL_CELL_TYPE_1,B_CELL,DUCTAL_CELL_TYPE_1,DUCTAL_CELL_TYPE_1,DUCTAL_CELL_TYPE_1
5,peng_seurat,6,DUCTAL_CELL_TYPE_2,DUCTAL_CELL_TYPE_2,B_CELL,DUCTAL_CELL_TYPE_2,DUCTAL_CELL_TYPE_2,DUCTAL_CELL_TYPE_2
6,peng_seurat,7,ENDOTHELIAL_CELL,ENDOTHELIAL_CELL,ACINAR_CELL,ENDOTHELIAL_CELL,ENDOTHELIAL_CELL,ENDOTHELIAL_CELL
7,peng_seurat,8,FIBROBLAST,FIBROBLAST,B_CELL,FIBROBLAST,FIBROBLAST,FIBROBLAST
8,peng_seurat,9,DUCTAL_CELL_TYPE_1,DUCTAL_CELL_TYPE_1,ACINAR_CELL,DUCTAL_CELL_TYPE_1,DUCTAL_CELL_TYPE_1,DUCTAL_CELL_TYPE_1
9,peng_seurat,10,T_CELL,T_CELL,B_CELL,T_CELL,T_CELL,T_CELL
